In [1]:
# !pip install -U pandas altair sqlalchemy

In [2]:
from sqlalchemy import create_engine
import pandas as pd
import altair as alt

# for inline display
from IPython.display import Image, HTML

'2022-02-03 10 AM'

In [9]:

engine = create_engine('sqlite:///current_orders.db')

sql = """\
SELECT
ifnull(round(
    ((fund.expenditure + fund.encumbrance)*1.0 / (NULLIF(appropriation, 0) * 1.0))*100.0,
    2
), 0) as total_percent_fund_used,
ifnull(round(
    ((fund.expenditure)*1.0 / (NULLIF(appropriation, 0) * 1.0))*100.0,
    2
), 0) as " expenditure % used",
ifnull(round(
    ((fund.encumbrance)*1.0 / (NULLIF(appropriation, 0) * 1.0))*100.0,
    2
), 0) as "encumbrance % used",
coalesce(external_fund_name, '') 
    || ' / ' 
    || coalesce(fund_code, '') as "fund name / fund code",
fund_code,
appropriation,
encumbrance,
expenditure
-- *
FROM
fund
WHERE
fund.fund_type = 'fbal'
AND appropriation > 0
-- ORDER BY percent_fund_used DESC
"""

df = pd.read_sql(sql=sql, con=engine)
# df.head()

# melt "expenditure_percent_fund_used" "encumbrance_percent_fund_used" into "fund use type" ...
out_df = df.melt(
    id_vars=["total_percent_fund_used", "fund name / fund code", "appropriation", "encumbrance", "expenditure"],  
    value_vars=[" expenditure % used", "encumbrance % used"],
    var_name="fund use type", 
    value_name="percent"
)

out_df.head()

# place the chart into a variable
chart = alt.Chart(out_df).mark_bar().encode(    
    x=alt.X('percent',),
    y=alt.Y('fund name / fund code', sort='-x'),
    color='fund use type',
    tooltip=['total_percent_fund_used', 'fund name / fund code', 'appropriation', 'encumbrance', 'expenditure', 'fund use type', 'percent']
    # tooltip=['percent_fund_used', 'fund_name', 'fund_code', 'appropriation', 'encumbrance', 'expenditure']
    # tooltip=['transaction_date', 'transaction_location_code', 'op_name', 'count'],
    #color='op_name'
).properties(
    title = 'Fund Use ( as of {} )'.format(pd.Timestamp('now').strftime('%Y-%m-%d %I:00 %p'))
    #title='Total Checkouts Per Location - {}'.format(search_date)
).configure_title(
    # fontSize=20,
    # anchor='start',
)

# display the chart
chart.save('output.html')
HTML('output.html')